In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import argrelextrema
import numpy as np
from datetime import datetime, timedelta
from pandas.core.common import SettingWithCopyWarning
import warnings

In [2]:
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [3]:
df = pd.read_csv('../data_file/isolate_merge_df.csv')

In [4]:
test_df = pd.read_csv('../data_file/fill_na_test.csv')

### holiday 37부터 43은 이상한 양상을 띈다. 기존 holiday feature를 극솟값에 대해서 0이되게 수정

In [5]:
def find_holiday(df, i):
    building_df = df[df['building_number'] == i]
    building_df['date_time'] = pd.to_datetime(building_df['date_time'], format='%Y%m%d %H')
    building_df.set_index('date_time', inplace=True)
    building_df_daily_max = building_df.resample('D').max()
    # 평균
    avg_pwr = (building_df_daily_max['power_consumption'].max() + building_df_daily_max['power_consumption'].min())/2
    
    # local minima 찾기
    comparator = np.less
    local_minima_indices = argrelextrema(building_df_daily_max['power_consumption'].values, comparator)
    local_minima = building_df_daily_max['power_consumption'].iloc[local_minima_indices]
    local_minima = local_minima[local_minima < avg_pwr]
    #print(local_minima)
    
    building_df['holiday'] = 0
    for i in local_minima.index:
        building_df.loc[building_df.index[(building_df.index >= i) & (building_df.index  < (i + timedelta(days=1)))], 'holiday'] =1
    
    df[df['building_number'] == i]['holiday'] = building_df['holiday']
    return df
    
    

### test set은 다 0으로 수정(휴일없음)

In [6]:
for i in range(37, 43):
    test_df[test_df['building_number'] == i]['holiday'] = 0

### train set 수정

In [7]:
for i in range(37, 43):
    df = find_holiday(df, i)

In [8]:
test_df.to_csv('../data_file/fix_holiday_feature_test.csv', index=False)

In [9]:
df.to_csv('../data_file/fix_holiday_feature_train.csv', index=False)